In [1]:
import numpy

In [2]:
import torch
import torch.utils.data
from torch import optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.utils import save_image

In [3]:
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [4]:
from pred_learn.models.vae_wm import VAE
from pred_learn.data import EnvDataset

In [5]:
import gym_ple

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


/home/ira/code/cool-code/gym/gym/envs/registration.py:56: UserWarning: DEPRECATION WARNING wrapper_config.TimeLimit has been deprecated. Replace any calls to `register(tags={'wrapper_config.TimeLimit.max_episode_steps': 200)}` with `register(max_episode_steps=200)`. This change was made 2017/1/31 and is included in gym version 0.8.0. If you are getting many of these warnings, you may need to update switch from universe 0.21.3 to retro (https://github.com/openai/retro)
  warnings.warn("DEPRECATION WARNING wrapper_config.TimeLimit has been deprecated. Replace any calls to `register(tags={'wrapper_config.TimeLimit.max_episode_steps': 200)}` with `register(max_episode_steps=200)`. This change was made 2017/1/31 and is included in gym version 0.8.0. If you are getting many of these warnings, you may need to update switch from universe 0.21.3 to retro (https://github.com/openai/retro)")


In [6]:
ENV_ID = "Snake-ple-v0"
action_space_n = gym_ple.make(ENV_ID).action_space.n
dataset_train = EnvDataset("../recorded/{}/0.torch".format(ENV_ID), action_space_n)
train_loader = torch.utils.data.DataLoader(
    dataset_train, batch_size=32, shuffle=True, num_workers=2
)

/home/ira/code/cool-code/gym/gym/logger.py:30: UserWarning: WARN: Environment '<class 'gym_ple.ple_env.PLEEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
len(dataset_train.record)

10082

In [8]:
action_space_n

5

In [18]:
# replay record
with torch.no_grad():
    for i in range(1000):
        timestep = dataset_train[i]
        obs = timestep["s0"]
        clear_output(True)
        plt.subplot(121)
        plt.imshow(obs.permute([2,1,0]))
    #     obs = resize(obs, (64, 64))
        plt.subplot(122)
        obs = model(timestep["s0"].unsqueeze(0).cuda())[0].squeeze(0).cpu()
        plt.imshow(obs.permute([2,1,0]))

    #     plt.imshow(timestep["s0"])
#         print("reward", timestep['r1'])
#         print("action", timestep['a0'])
        plt.pause(0.02)

KeyboardInterrupt: 

In [11]:
obs.max()

tensor(1.)

In [12]:
model = VAE(3, 32).cuda()
optimizer = optim.Adam(model.parameters())

In [13]:
def loss_function(recon_x, x, mu, logsigma):
    """ VAE loss function """
    MSE = F.mse_loss(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + 2 * logsigma - mu.pow(2) - (2 * logsigma).exp())
    return MSE + KLD

In [17]:
for epoch in range(1000):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        data = data["s0"].cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

Train Epoch: 0 [0/10082 (0%)]	Loss: 39.632984
Train Epoch: 0 [640/10082 (6%)]	Loss: 39.937458
Train Epoch: 0 [1280/10082 (13%)]	Loss: 38.684395
Train Epoch: 0 [1920/10082 (19%)]	Loss: 41.371075
Train Epoch: 0 [2560/10082 (25%)]	Loss: 39.613411
Train Epoch: 0 [3200/10082 (32%)]	Loss: 34.899521
Train Epoch: 0 [3840/10082 (38%)]	Loss: 39.402260
Train Epoch: 0 [4480/10082 (44%)]	Loss: 37.968224
Train Epoch: 0 [5120/10082 (51%)]	Loss: 40.301479
Train Epoch: 0 [5760/10082 (57%)]	Loss: 38.330399
Train Epoch: 0 [6400/10082 (63%)]	Loss: 39.654491
Train Epoch: 0 [7040/10082 (70%)]	Loss: 33.359291
Train Epoch: 0 [7680/10082 (76%)]	Loss: 40.076916
Train Epoch: 0 [8320/10082 (82%)]	Loss: 36.115391
Train Epoch: 0 [8960/10082 (89%)]	Loss: 38.373619
Train Epoch: 0 [9600/10082 (95%)]	Loss: 37.541954
====> Epoch: 0 Average loss: 38.3000
Train Epoch: 1 [0/10082 (0%)]	Loss: 36.130283
Train Epoch: 1 [640/10082 (6%)]	Loss: 38.180878
Train Epoch: 1 [1280/10082 (13%)]	Loss: 35.312771
Train Epoch: 1 [1920/1008

Train Epoch: 9 [9600/10082 (95%)]	Loss: 23.224014
====> Epoch: 9 Average loss: 25.1976
Train Epoch: 10 [0/10082 (0%)]	Loss: 23.352974
Train Epoch: 10 [640/10082 (6%)]	Loss: 24.455082
Train Epoch: 10 [1280/10082 (13%)]	Loss: 22.372635
Train Epoch: 10 [1920/10082 (19%)]	Loss: 23.127413
Train Epoch: 10 [2560/10082 (25%)]	Loss: 22.433331
Train Epoch: 10 [3200/10082 (32%)]	Loss: 21.971769
Train Epoch: 10 [3840/10082 (38%)]	Loss: 23.102783
Train Epoch: 10 [4480/10082 (44%)]	Loss: 22.961426
Train Epoch: 10 [5120/10082 (51%)]	Loss: 23.150261
Train Epoch: 10 [5760/10082 (57%)]	Loss: 26.123680
Train Epoch: 10 [6400/10082 (63%)]	Loss: 23.885637
Train Epoch: 10 [7040/10082 (70%)]	Loss: 23.600716
Train Epoch: 10 [7680/10082 (76%)]	Loss: 26.485451
Train Epoch: 10 [8320/10082 (82%)]	Loss: 22.280029
Train Epoch: 10 [8960/10082 (89%)]	Loss: 23.118330
Train Epoch: 10 [9600/10082 (95%)]	Loss: 21.853273
====> Epoch: 10 Average loss: 23.7878
Train Epoch: 11 [0/10082 (0%)]	Loss: 24.540503
Train Epoch: 11 [6

KeyboardInterrupt: 

In [ ]:
data

In [ ]:
net(timestep["s1"].permute([2, 1, 0]).unsqueeze(0))

In [ ]:
timestep["s1"].permute([2, 1, 0]).unsqueeze(0).size()